# P-values in ML 02

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sc
import sklearn as sk
import sys
import os

from scipy.stats import chi2
from sklearn.ensemble import RandomForestClassifier
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.multitest import fdrcorrection

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Create dataset

In [2]:
def logit(h):
    ''' Logistic from activation h '''
    p = 1.0 / (1.0 + np.exp(-h))
    r = np.random.rand(len(p))
    y = (r < p).astype('float')
    return y


def rand_date():
    max_time = int(time.time())
    t = random.randint(0, max_time)
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))


def rand_unif(num, mean, std, na_prob=0):
    xi = np.random.rand(num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rand_norm(num, mean=0.0, std=1.0, na_prob=0):
    xi = np.random.normal(mean, std, num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rbf(x, mu=0, sigma=1):
    """ Radial basis function """
    z = (x - mu) / sigma
    return np.exp(-(z*z))


def split_df(df, var_inputs, var_output, train_index, val_index):
    df_train, df_val = df.iloc[train_index], df.iloc[val_index]
    return df_train[var_inputs], df_train[var_output], df_val[var_inputs], df_val[var_output]

In [3]:
def create_dataset_01(num=1000, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    x4 = rand_norm(num)
    x5 = rand_norm(num)
    n = rand_norm(num)
    y = logit(2.0 * x1 - 1.0 * x2 + 0.5 * x3 + 0.25 * n)
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6, 'y': y})
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

In [4]:
def create_dataset_02(num=1000, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    x4 = rand_norm(num)
    x5 = rand_norm(num)
    x6 = rand_norm(num)
    n = rand_norm(num)
    y = logit(2.0 * rbf(x1, 1, 1) - 1.0 * rbf(x2, -1, 1) + 0.5 * x3 + 0.25 * n)
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6, 'y': y})
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

## Feature importance: Logistic regression

In [5]:
def wilks_model_fit(x, y):
    logit_model = Logit(y, x)
    res = logit_model.fit(disp=0)
    return logit_model, res

def wilks_p_value(df, var_output, vars_null):
    model_null, model_null_results = wilks_model_fit(df[vars_null], df[var_output])
    pvalues = dict()
    for c in df.columns:
        if c != var_output and c not in vars_null:
            xnames = list(vars_null)
            xnames.append(c)
            model_alt, model_alt_res = wilks_model_fit(df[xnames], df[var_output])
            if model_alt is None:
                self._error(f"Could not fit alt model for column/s {c}")
                pval = 1.0
            else:
                d = 2.0 * (model_alt_res.llf - model_null_results.llf)
                pval = chi2.sf(d, 1)
            pvalues[c] = pval
    return pd.Series(pvalues)

## Feature importance: ML

### Cross-validations & models

In [6]:
from sklearn.model_selection import KFold


def new_model_fit(x_train, y_train):
    model = RandomForestClassifier(100)
    model.fit(x_train, y_train)
    return model

_id = 0

def reset_id():
    _id = 0

def next_id():
    global _id
    _id += 1
    return _id


def is_var_rand(var):
    return var is not None and var.startswith('__rand_')


class FeatureImportanceShuffle:
    """ Feature importance by shuffling input values """
    def __init__(self, model, x_train, y_train, x_val, y_val, num_iter=10):
        self.id = next_id()
        self.model = model
        self.x_train, self.y_train, self.x_val, self.y_val = x_train, y_train, x_val, y_val
        self.num_iter = num_iter
        self.score_null = None  # Null score (one score)
        self.scores_alt = dict()  # Alt scores by variable name (multiple scores for each variable)

    def add_score(self, var, score):
        """ Add score for variable 'var' """
        if var not in self.scores_alt:
            self.scores_alt[var] = list()
        self.scores_alt[var].append(score)

    def __call__(self):
        """ Feature importance by shuffling variable 'var' and comparing performance results """
        if not self.score_null:
            self.score_null = self.model.score(self.x_val, self.y_val)
            # print(f"\t\tFeature importance (id={self.id}): Null model {self.scores_null}")
        for var in self.x_train.columns:
            # Perform shuffling 'num_iter' times
            scores = list()
            for i in range(self.num_iter):
                score = self.feature_importance_variable(var)
                scores.append(score)
                self.add_score(var, score)
            scores = np.array(scores)
            # print(f"\t{var}:\tcount: {len(scores)}\tmean: {scores.mean()}\tstd: {scores.std()}")

    def feature_importance_variable(self, var):
        """ Feature importance by shuffling variable 'var' and comparing performance results """
        # Shuffle 'var' and calculate validation score
        x_shuf = self.x_val.copy()
        x_shuf[var] = x_shuf[var].sample(frac=1).values
        score = self.model.score(x_shuf, self.y_val)
        # print(f"\t\tFeature importance (id={self.id}): Variable '{var}, score: {score}'")
        return score

    def get_null(self):
        """ Get null score """
        return self.score_null

    def get_null_and_rand(self, exclude_var=None):
        """ Get null and all '__rand_' scores as a list """
        values = [self.score_null]
        for var in self.x_train.columns:
            if is_var_rand(var) and var != exclude_var:
                values.extend(self.scores_alt[var])
        return values

    def get_alt(self, var):
        """ Get scores for variable 'var', or None if not found """
        return self.scores_alt.get(var)


class FeatureImportance:
    """
    Calculate feature importance.
    """
    def __init__(self, df, var_output, vars_input=None, vars_null=None, new_model_function=new_model_fit, num_cv=5, num_shuffle=10, num_rand=0, max_iter=100):
        self.df_ori = df
        self.var_output, self.vars_input, self.vars_null = var_output, vars_input, vars_null
        if self.vars_input is None:
            self.vars_input = [c for c in df.columns if c != var_output]
        if self.vars_null is None:
            self.vars_null = [c for c in self.vars_input if c.startswith('rand_')]
        self.new_model_function = new_model_function
        self.df = None
        self.pvals_df = None
        self.max_iter =max_iter
        self.num_cv = num_cv
        self.num_rand = num_rand
        self.num_shuffle = num_shuffle
        self.cvs = list()  # List of cross validation items
    
    def __call__(self):
        # initialize
        self.df = self.extend_df_ori()
        self.initialize_cv()
        # Calculate importance by shuffling
        for count_iter in range(self.max_iter):
            self.pvals_df = pd.DataFrame()
            for fis in self.cvs:
                print(f"Iteration: {count_iter}, Cross-validation: {fis.id}")
                fis()
            stop = self.p_values()
            if stop:
                return self.pvals_df
        return self.pvals_df
    
    def cv_iter(self, df):
        """ Create a cross-validation iterator """
        if self.num_cv > 1:
            return KFold(n_splits=self.num_cv).split(df)
        else:
            # No cross validation, split 80% / 20%
            idx = int(0.8 * len(df))
            idx_train = range(0, idx)
            idx_val = range(idx, len(df))
            return [(idx_train, idx_val)]
        
    def extend_df_ori(self):
        """ Extend the data frame by adding random permutations and random variables """
        df = self.df_ori.copy()
        # Create one additional 'rand' column for every variable
        for c in self.df_ori.columns:
            if c == self.var_output:
                continue
            c_rand = f"__rand_{c}"
            self.vars_input.append(c_rand)
            df[c_rand] = df[c].sample(frac=1).values
        # Create one additional 'rand' column for every variable
        for i in range(self.num_rand):
            c_rand = f"__rand_{i}"
            self.vars_input.append(c_rand)
            df[c_rand] = np.random.rand(len(df[c]))
        return df
        
    def get_alts(self, var):
        """ Get alt scores for variable 'var' """
        scores = list()
        for fis in self.cvs:
            ss = fis.get_alt(var)
            if ss is not None:
                scores.extend(ss)
        return np.array(scores)

    def get_nulls(self, var):
        """ Get null scores, excluding variable 'var' """
        scores = list()
        for fis in self.cvs:
            ss = fis.get_null_and_rand(exclude_var=var)
            if ss is not None:
                scores.extend(ss)
        return np.array(scores)

    def initialize_cv(self):
        """ Initialize cross validations """
        reset_id()
        for train_index, val_index in self.cv_iter(self.df):
            x_train, y_train, x_validate, y_validate = split_df(self.df, self.vars_input, self.var_output, train_index, val_index)
            model = self.new_model_function(x_train, y_train)
            fis = FeatureImportanceShuffle(model, x_train, y_train, x_validate, y_validate, num_iter=self.num_shuffle)
            self.cvs.append(fis)

    def p_value(self, var, null_scores, alt_scores):
        null_scores, alt_scores = np.array(null_scores), np.array(alt_scores)
        pval_greater = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='greater')[1]
        pval_less = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='less')[1]
        pval_two = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='two-sided')[1]
        df_row = pd.DataFrame({'name': var
                               , 'num_samples': len(self.df_ori)
                               , 'num_cv': self.num_cv
                               , 'num_shuffle': self.num_shuffle
                               , 'count_alt': len(alt_scores)
                               , 'mean_alt': alt_scores.mean()
                               , 'std_alt': alt_scores.std()
                               , 'count_null': len(null_scores)
                               , 'mean_null': null_scores.mean()
                               , 'std_null': null_scores.std()
                               , 'p_value': pval_greater
                               , 'p_value_less': pval_less
                               , 'pval_two_sided': pval_two
                              }, index=[len(self.pvals_df)])
        self.pvals_df = self.pvals_df.append(df_row)
        return pval_greater

    def p_values(self):
        # Get null scores
        null_scores = list()
        for fis in self.cvs:
            null_scores.extend(fis.get_null_and_rand())
        # Calculate p-value for every variable
        pvals, isrand = list(), list()
        for var in self.vars_input:
            alt_scores = self.get_alts(var)
            null_scores = self.get_nulls(var)
            pval = self.p_value(var, null_scores, alt_scores)
            pvals.append(pval)
            isrand.append(is_var_rand(var))
        pvals = np.array(pvals)
        isrand = np.array(isrand)
        rejected, pvals_fdr = fdrcorrection(pvals)
        self.pvals_df['is_rand'] = isrand
        self.pvals_df['pvalues_fdr'] = pvals_fdr
        self.pvals_df['pvalues_fdr_reject'] = rejected
        return (rejected & isrand).any()


### P-values analysis

In [7]:
num_samples = 300
df = create_dataset_02(num_samples)
df.head()

,x1,x2,x3,x4,x5,x6,y
0,1.308437,0.075794,0.552599,-0.574779,1.140167,-0.942430,0.0
1,-2.028171,0.153609,-0.490268,1.741397,-0.155111,1.299311,0.0
2,-0.941189,-0.207906,0.365694,-1.464444,0.758382,-1.061532,0.0
3,0.729480,-1.739190,1.278568,0.411908,-0.867971,1.194513,1.0
4,1.514457,0.184485,1.625167,-1.382087,0.043655,0.749920,0.0


In [8]:
def pvalues_shuffle_analysis():
    pvals_df = None
    for dataset_num in [1, 2]:
        for num_samples in [50, 100, 200, 300, 500, 1000, 2000, 5000, 10000]:
            for num_cv in [1, 3, 5, 10, 20]:
                for num_shuffle in [3, 5, 10, 20, 50, 100]:
                    print(f"dataset_num: {dataset_num}\tnum_samples: {num_samples}\tnum_cv: {num_cv}\tnum_shuffle: {num_shuffle}")
                    df = create_dataset_01(num_samples) if dataset_num == 1 else create_dataset_02(num_samples)
                    fi = FeatureImportance(df, 'y', num_shuffle=num_shuffle, num_cv=num_cv)
                    pvals_df = fi()
                    pvals_df.to_csv(f"pvals_df.dataset_num_{dataset_num}.num_samples_{num_samples}.num_cv_{num_cv}.num_shuffle_{num_shuffle}.csv", index=False)
                    display(pvals_df)

                
pvalues_shuffle_analysis()

num_samples:50	num_cv: 1	num_shuffle:3
Iteration: 0, Cross-validation: 1
Iteration: 1, Cross-validation: 1
Iteration: 2, Cross-validation: 1
Iteration: 3, Cross-validation: 1
Iteration: 4, Cross-validation: 1


,name,num_samples,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value,p_value_less,pval_two_sided,is_rand,pvalues_fdr,pvalues_fdr_reject
0,x1,50,1,3,15,0.433333,6.992059e-02,91,0.436264,0.071892,0.574441,4.301619e-01,8.603237e-01,False,0.765922,False
1,x2,50,1,3,15,0.400000,1.110223e-16,91,0.436264,0.071892,0.017509,9.830375e-01,3.501838e-02,False,0.052528,False
2,x3,50,1,3,15,0.413333,3.399346e-02,91,0.436264,0.071892,0.153071,8.498414e-01,3.061427e-01,False,0.306143,False
3,x4,50,1,3,15,0.486667,9.568467e-02,91,0.436264,0.071892,0.996800,3.308908e-03,6.617817e-03,False,1.000000,False
4,x5,50,1,3,15,0.406667,2.494438e-02,91,0.436264,0.071892,0.059815,9.416679e-01,1.196300e-01,False,0.143556,False
5,x6,50,1,3,15,0.460000,7.118052e-02,91,0.436264,0.071892,0.950747,5.044590e-02,1.008918e-01,False,1.000000,False
6,__rand_x1,50,1,3,15,0.426667,4.422166e-02,76,0.438158,0.076032,0.497151,5.085473e-01,9.943014e-01,True,0.745726,False
7,__rand_x2,50,1,3,15,0.426667,4.422166e-02,76,0.438158,0.076032,0.497151,5.085473e-01,9.943014e-01,True,0.745726,False
8,__rand_x3,50,1,3,15,0.566667,7.888106e-02,76,0.410526,0.030689,1.000000,2.094766e-13,4.189532e-13,True,1.000000,False
9,__rand_x4,50,1,3,15,0.400000,1.110223e-16,76,0.443421,0.076667,0.009392,9.909619e-01,1.878498e-02,True,0.037570,True


num_samples:50	num_cv: 1	num_shuffle:5
Iteration: 0, Cross-validation: 2
Iteration: 1, Cross-validation: 2


,name,num_samples,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value,p_value_less,pval_two_sided,is_rand,pvalues_fdr,pvalues_fdr_reject
0,x1,50,1,5,10,0.38,0.074833,61,0.414754,0.072057,0.068197,0.934175,0.136394,False,0.204591,False
1,x2,50,1,5,10,0.40,0.000000,61,0.414754,0.072057,0.147881,0.856505,0.295763,False,0.313580,False
2,x3,50,1,5,10,0.45,0.050000,61,0.414754,0.072057,0.925891,0.076745,0.153489,False,0.997886,False
3,x4,50,1,5,10,0.35,0.067082,61,0.414754,0.072057,0.005242,0.995030,0.010484,False,0.031452,True
4,x5,50,1,5,10,0.40,0.044721,61,0.414754,0.072057,0.182922,0.822029,0.365843,False,0.313580,False
5,x6,50,1,5,10,0.38,0.074833,61,0.414754,0.072057,0.068197,0.934175,0.136394,False,0.204591,False
6,__rand_x1,50,1,5,10,0.40,0.063246,51,0.417647,0.073313,0.182405,0.823316,0.364810,True,0.313580,False
7,__rand_x2,50,1,5,10,0.47,0.045826,51,0.403922,0.071293,0.997886,0.002264,0.004528,True,0.997886,False
8,__rand_x3,50,1,5,10,0.46,0.048990,51,0.405882,0.072523,0.988755,0.011906,0.023813,True,0.997886,False
9,__rand_x4,50,1,5,10,0.35,0.067082,51,0.427451,0.065913,0.001245,0.998842,0.002491,True,0.014946,True


num_samples:50	num_cv: 1	num_shuffle:10
Iteration: 0, Cross-validation: 3
Iteration: 1, Cross-validation: 3


...

,name,num_samples,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value,p_value_less,pval_two_sided,is_rand,pvalues_fdr,pvalues_fdr_reject
0,x1,50,10,10,200,0.532,0.219490,1210,0.567438,0.238932,0.034997,0.965018,0.069995,False,0.209984,False
1,x2,50,10,10,200,0.571,0.247707,1210,0.567438,0.238932,0.590789,0.409287,0.818573,False,0.982129,False
2,x3,50,10,10,200,0.582,0.241818,1210,0.567438,0.238932,0.860414,0.139629,0.279259,False,0.982129,False
3,x4,50,10,10,200,0.597,0.226696,1210,0.567438,0.238932,0.907698,0.092334,0.184668,False,0.982129,False
4,x5,50,10,10,200,0.599,0.217713,1210,0.567438,0.238932,0.978619,0.021391,0.042783,False,0.982129,False
5,x6,50,10,10,200,0.607,0.261057,1210,0.567438,0.238932,0.982129,0.017879,0.035759,False,0.982129,False
6,__rand_x1,50,10,10,200,0.562,0.237815,1010,0.568515,0.239138,0.508563,0.491529,0.983058,True,0.982129,False
7,__rand_x2,50,10,10,200,0.566,0.229878,1010,0.567723,0.240683,0.499171,0.500921,0.998342,True,0.982129,False
8,__rand_x3,50,10,10,200,0.529,0.234860,1010,0.575050,0.238998,0.003287,0.996715,0.006574,True,0.039447,True
9,__rand_x4,50,10,10,200,0.595,0.244489,1010,0.561980,0.237437,0.967544,0.032473,0.064946,True,0.982129,False


num_samples:50	num_cv: 10	num_shuffle:20
Iteration: 0, Cross-validation: 85
Iteration: 0, Cross-validation: 86
Iteration: 0, Cross-validation: 87
Iteration: 0, Cross-validation: 88
Iteration: 0, Cross-validation: 89
Iteration: 0, Cross-validation: 90
Iteration: 0, Cross-validation: 91
Iteration: 0, Cross-validation: 92
Iteration: 0, Cross-validation: 93
Iteration: 0, Cross-validation: 94
Iteration: 1, Cross-validation: 85
Iteration: 1, Cross-validation: 86
Iteration: 1, Cross-validation: 87
Iteration: 1, Cross-validation: 88
Iteration: 1, Cross-validation: 89
Iteration: 1, Cross-validation: 90
Iteration: 1, Cross-validation: 91
Iteration: 1, Cross-validation: 92
Iteration: 1, Cross-validation: 93
Iteration: 1, Cross-validation: 94
Iteration: 2, Cross-validation: 85
Iteration: 2, Cross-validation: 86
Iteration: 2, Cross-validation: 87
Iteration: 2, Cross-validation: 88
Iteration: 2, Cross-validation: 89
Iteration: 2, Cross-validation: 90
Iteration: 2, Cross-validation: 91
Iteration: 2, 

,name,num_samples,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value,p_value_less,pval_two_sided,is_rand,pvalues_fdr,pvalues_fdr_reject
0,x1,50,10,20,2200,0.546545,0.178725,13210,0.582377,0.164046,6.573801e-14,1.000000,1.314760e-13,False,7.888561e-13,True
1,x2,50,10,20,2200,0.586273,0.190246,13210,0.582377,0.164046,2.774980e-01,0.722504,5.549960e-01,False,4.162470e-01,False
2,x3,50,10,20,2200,0.541364,0.208627,13210,0.582377,0.164046,8.057825e-13,1.000000,1.611565e-12,False,4.834695e-12,True
3,x4,50,10,20,2200,0.575182,0.189982,13210,0.582377,0.164046,1.986383e-01,0.801363,3.972766e-01,False,3.405228e-01,False
4,x5,50,10,20,2200,0.584909,0.158486,13210,0.582377,0.164046,4.667355e-01,0.533267,9.334711e-01,False,6.223140e-01,False
5,x6,50,10,20,2200,0.562364,0.151843,13210,0.582377,0.164046,3.138028e-08,1.000000,6.276056e-08,False,1.255211e-07,True
6,__rand_x1,50,10,20,2200,0.580455,0.170133,11010,0.582761,0.162800,6.757380e-02,0.932427,1.351476e-01,True,1.351476e-01,False
7,__rand_x2,50,10,20,2200,0.573273,0.174601,11010,0.584196,0.161793,2.653354e-02,0.973467,5.306708e-02,True,6.368049e-02,False
8,__rand_x3,50,10,20,2200,0.585091,0.157467,11010,0.581835,0.165324,9.225911e-01,0.077410,1.548198e-01,True,9.968504e-01,False
9,__rand_x4,50,10,20,2200,0.588545,0.168565,11010,0.581144,0.163101,9.252278e-01,0.074773,1.495462e-01,True,9.968504e-01,False


num_samples:50	num_cv: 10	num_shuffle:50
Iteration: 0, Cross-validation: 95
Iteration: 0, Cross-validation: 96
Iteration: 0, Cross-validation: 97
Iteration: 0, Cross-validation: 98
Iteration: 0, Cross-validation: 99
Iteration: 0, Cross-validation: 100
Iteration: 0, Cross-validation: 101
Iteration: 0, Cross-validation: 102


KeyboardInterrupt: 

# Improvement: Cluster high correlations (groups of variables)